Dans ce notebook, on indique la démarche suivie pour créer une base de donnée prête à être analysée à partir des indicateurs issus de la banque mondiale (https://data.worldbank.org/indicator/). On peut reprendre cette démarche pour rajouter ou enlever d'autres indicateurs, et on peut à la fin choisir de ne conserver que certains pays en en mettant la liste. 

La base créée dans ce code comporte 5 indicateurs (PIB, PIB_HAB, FBCF en $, Tx de chômage et Exportations en $). On dispose de ces données complètes entre 2000 et 2018 pour 145 pays

# Code

On commence par importer les modules nécessaires, et on crée quelques foncions qui vont nous aider à manipuler les dataframe en jonglant entre les codes et les noms des pays

In [109]:
import pandas as pd
import re
import numpy as np

def refresh_dico(Name,Code,df):
    dico = {}
    for i in df[[Name,Code]].dropna().drop_duplicates().iterrows() :
        dico[i[1][Name]] = i[1][Code]
    return (dico)

def rech_ligne_pays (y,df) :
    expression = re.compile(dic_pays[y]+".*?;")
    return(list(i[:-1] for i in (expression.findall(';'.join(df.index)+';'))))

def rech_ligne_indic (y,df):
    expression = re.compile(".{,4}"+y+";")
    return(list(i[:-1] for i in (expression.findall(';'.join(df.index)+';'))))

On importe ensuite les bases au format csv, que l'on est allé chercher sur le site de la banque mondiale qui est en open data (https://data.worldbank.org/indicator/).

Attention, pour ouvrir proprement ces données, il faut d'abord ouvrir les csv dans le bloc notes (ou n'importe quel autre logiciel capable de les ouvrir), et supprimer les deux premières lignes du document qui correspondent plus ou moins au titre du document, mais qui font bugger la fonction pd.read_csv.

In [127]:
#Importation des bases csv, et crétaion de la base aggrégée
pib = pd.read_csv('pib.csv')
pib_hab = pd.read_csv('pib_hab.csv')
fbcf = pd.read_csv('FBCF.csv')
exports_rate = pd.read_csv('exports_rate.csv')
unemployment_rate = pd.read_csv('unemployment_rate.csv')

#création de la base aggrégée
df_list = [pib, pib_hab, fbcf, unemployment_rate, exports_rate]

for df in df_list :
    df['Pays_indic'] = df['Country Code'].str[:3] + '_' + df['Indicator Name'].str[:99]

df1 = pd.concat(df_list)
df1 = df1.sort_values('Pays_indic').set_index('Pays_indic')

On supprime d'abord les colonnes qui ne nous servent à rien : le nom de code de l'indicateur, les années que l'on ne conserve pas pour l'étude, et la dernière colonne qui est un bug lors de l'ouverture de ces données avec la fonction pd.read_csv

On fait ensuite de même avec les lignes, en supprimant tous les aggrégats de pays qui ne sont pas proprement des pays

In [128]:
#SUPPRESSION DES LIGNES ET COLONNES INUTILES
dic_pays = refresh_dico('Country Name','Country Code',df1)


useless_data_list = ['Unnamed: 65', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', 
                   '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', 
                   '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
                   '1995', '1996', '1997', '1998', '1999', '2019', '2020']
df1 = df1.drop(useless_data_list, axis = 1)


suppr_liste = ['Arab World', 'Central Europe and the Baltics',
 'Caribbean small states', 'East Asia & Pacific (excluding high income)', 
  'Early-demographic dividend', 
 'East Asia & Pacific',
 'Europe & Central Asia (excluding high income)', 
 'Europe & Central Asia',  
 'European Union','Fragile and conflict affected situations',
 'High income',
 'Heavily indebted poor countries (HIPC)','IBRD only',
 'IDA & IBRD total',
 'IDA total',
 'IDA blend',
 'IDA only',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & Caribbean',
 'Least developed countries: UN classification',
 'Low income',
 'Lower middle income',
 'Low & middle income',
 'Late-demographic dividend',
 'Middle East & North Africa',
 'Middle income',
 'Middle East & North Africa (excluding high income)',
 'North America',
 'OECD members',
 'Other small states',
 'Pre-demographic dividend',
 'Pacific island small states',
 'Post-demographic dividend',
 'South Asia',
 'Sub-Saharan Africa (excluding high income)',
 'Sub-Saharan Africa',
 'Small states',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'South Asia (IDA & IBRD)',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Upper middle income',
 'World',
 'Central African Republic',
 'West Bank and Gaza',
 'Hong Kong SAR, China',
 'Macao SAR, China']
for i in suppr_liste :
    df1 = df1.drop(labels = rech_ligne_pays(i,df1)) 

On supprime maintenant tous les pays pour lesquels il manque au moins un indicateur. On le fait en deux étapes :

(1) On supprime toutes les lignes où il manque au moins une donnée à un moment donné.
(2) On supprime toutes les lignes de tous les pays pour lesquels on a supprimé au moins une ligne

In [134]:
#Étape 1
df1 = df1.dropna(axis = 0)
df1.head(7)

#Étape 2 
dic_code = refresh_dico('Country Code','Country Name',df1)

for i in dic_pays.keys() :
    if len(rech_ligne_pays(i,df1))<5 :
        df1 = df1.drop(rech_ligne_pays(i,df1))

dic_code = refresh_dico('Country Code','Country Name',df1)

#On affiche le nombre de pays restant dans la base
print(len(dic_code.keys()))


145


On recrée maintenant la variable correpondant aux exportations par pays en dollars, à partir du PIB et des exportations en pourcentage du PIB. Après l'avoir fait, on supprime l'ancienne variable qui exprime les exportations en pct du PIB.
On a utlisé la formule :

    Exportation$ = (Exportations%PIB * PIB)/100

In [131]:
#CREATION DE LA VARIABLE EXPORTATION A PARTIR DE EXPORTATION RATE ET DU PIB

for i in dic_code.keys():
    df1.loc[i+ '_Exports of goods and services (current US$)']= (df1.loc[i+'_Exports of goods and services (% of GDP)'][3:].astype(np.float64) * df1.loc[i+'_GDP (current US$)'][3:].astype(np.float64)) / 100
    df1.loc[i+ '_Exports of goods and services (current US$)',['Country Name','Country Code']]=df1.loc[i+'_Exports of goods and services (% of GDP)',['Country Name','Country Code']]
    df1.loc[i+ '_Exports of goods and services (current US$)','Indicator Name']= 'Exports of goods and services (current US$)'
    df1 = df1.drop(labels = i+'_Exports of goods and services (% of GDP)')
df1 = df1.sort_index()

On affiche le nombre de pays, et on vérifie qu'on a bien toujours la France, et on affiche la taille du df.

In [139]:
print('On a conservé ', len(dic_code.keys()), ' pays')
print('La taille du df est de ', df1.shape)
df1.loc[rech_ligne_pays('France', df1)]


On a conservé  145  pays
La taille du df est de  (725, 22)


,Country Name,Country Code,Indicator Name,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Pays_indic,,,,,,,,,,,,,,,,,,,,,
FRA_Exports of goods and services (current US$),France,FRA,Exports of goods and services (current US$),3.895347e+11,3.890702e+11,4.113928e+11,4.805858e+11,5.600124e+11,5.936799e+11,6.477004e+11,...,6.681314e+11,7.079102e+11,8.132505e+11,7.837582e+11,8.254656e+11,8.461430e+11,7.459117e+11,7.475030e+11,8.031638e+11,8.842869e+11
FRA_GDP (current US$),France,FRA,GDP (current US$),1.362249e+12,1.376465e+12,1.494287e+12,1.840481e+12,2.115742e+12,2.196126e+12,2.318594e+12,...,2.690222e+12,2.642610e+12,2.861408e+12,2.683825e+12,2.811078e+12,2.852166e+12,2.438208e+12,2.471286e+12,2.595151e+12,2.787864e+12
FRA_GDP per capita (current US$),France,FRA,GDP per capita (current US$),2.236403e+04,2.243356e+04,2.417734e+04,2.956839e+04,3.374127e+04,3.476019e+04,3.644362e+04,...,4.157542e+04,4.063833e+04,4.379073e+04,4.087470e+04,4.259293e+04,4.301126e+04,3.663818e+04,3.703737e+04,3.881216e+04,4.163109e+04
FRA_Gross capital formation (% of GDP),France,FRA,Gross capital formation (% of GDP),2.248805e+01,2.216487e+01,2.131558e+01,2.118613e+01,2.188960e+01,2.245262e+01,2.323771e+01,...,2.133316e+01,2.194629e+01,2.322091e+01,2.262663e+01,2.228719e+01,2.270983e+01,2.271232e+01,2.260939e+01,2.343619e+01,2.387174e+01
"FRA_Unemployment, total (% of total labor force) (modeled ILO estimate)",France,FRA,"Unemployment, total (% of total labor force) (...",1.021700e+01,8.610000e+00,8.702000e+00,8.306000e+00,8.914000e+00,8.493000e+00,8.448000e+00,...,8.736000e+00,8.871000e+00,8.811000e+00,9.400000e+00,9.921000e+00,1.029200e+01,1.035900e+01,1.005700e+01,9.397000e+00,9.059000e+00


In [143]:
df1.to_csv('C:/Users/Titouan/Desktop/WB_data.csv')